<a href="https://colab.research.google.com/github/labrosse1/RAG-pipeline-Llamaindex/blob/Documents-ingestion/RAG_pipeline_Llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

!pip install python-dotenv

import os
from dotenv import load_dotenv

# Charger le fichier .env
load_dotenv('/content/drive/MyDrive/RAG-pipeline-Llamaindex/.env')

# Récupérer les clés
openai_key = os.environ.get('OPENAI_API_KEY')
scraping_key = os.environ.get('SCRAPING_BEE')

In [ ]:
!pip install scrapingbee

In [ ]:
from scrapingbee import ScrapingBeeClient

client = ScrapingBeeClient(api_key=scraping_key)


urls = ["https://docs.llamaindex.ai/en/stable/understanding/rag/",
        "https://docs.llamaindex.ai/en/stable/understanding/loading/loading/",
        "https://docs.llamaindex.ai/en/stable/understanding/indexing/indexing/",
        "https://docs.llamaindex.ai/en/stable/understanding/storing/storing/",
        "https://docs.llamaindex.ai/en/stable/understanding/querying/querying/",
        ];


for i, url in enumerate(urls, start=1):
    response = client.get(url)
    filename = f"scraped_page{i}.html"

    with open(filename, "w", encoding="utf-8") as f:
      f.write(response.text)
      print(f" {url} sauvegardée dans {filename}")

print('Response HTTP Status Code: ', response.status_code)

In [ ]:
!pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup

all_text = []

# Parcours des fichiers HTML
for file in os.listdir("."):
    if file.startswith("scraped_page") and file.endswith(".html"):
        print(f"Traitement de {file}...")

        with open(file, "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")

        # Supprimer scripts et styles
        for tag in soup(["script", "style", "noscript"]):
            tag.extract()

        # Construire un texte structuré pour cette page
        output_lines = []
        for element in soup.find_all(["h1", "h2", "h3", "p", "li"]):
            text = element.get_text(strip=True)
            if not text:
                continue
            if element.name == "h1":
                output_lines.append(f"# {text}\n")
            elif element.name == "h2":
                output_lines.append(f"## {text}\n")
            elif element.name == "h3":
                output_lines.append(f"### {text}\n")
            elif element.name == "li":
                output_lines.append(f"- {text}")
            else:
                output_lines.append(text + "\n")

        # Ajouter à la liste globale
        all_text.append("\n".join(output_lines))

# Joindre tout le texte en un seul bloc
final_text = "\n\n".join(all_text)

# Sauvegarde dans un fichier unique
with open("scraped_all.md", "w", encoding="utf-8") as f:
    f.write(final_text)